# Coin Detection Challenge - Image Analysis and Pattern Recognition - 2024
Lucille Niederhauser - Meriam Bouguecha - Raphaëlle Hartwig - Group 55


The goal of this project was to build an automatic system that can identify swiss franc and euro coins on an image with various backgrounds. To do so, we tackled the following tasks:
1. Background detection
2. Segmentation of coins for each background (neutral, noisy, hand)
3. Classifying single coins into CHF, EUR or OOD
4. Classifying CHF into their exact values
5. Classifying EUR into their exact values

In [141]:
# Import main packages & files
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
import numpy as np
import cv2
import os
from utils import *
from classify_background import *
from segmentation import *
from classification import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
# define paths used
original_train_path = './data/train/'
original_test_path = './data/test/'
train_path = './preprocessed_data/downsampled/train/'
test_path = './preprocessed_data/downsampled/test/'
cropped_path = './data/cropped/train/'

In [ ]:
# set the seeds for reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# 1. Background Detection
When starting to explore segmentation techniques, we quickly realised that it would be complicated to find one segmentation technique that is able to correctly indentify coins on all three backgrounds. We therefore decided to first create a function that given any image could find if the background was neutral, noisy or hand.

# 2. Coins Segmentation
Once we were able to detect the background, we developped three segmentation function for each background type. However, before doing so we decided to downsample our images because the original images are big and thus any processing of them takes a lot of time.

In [ ]:
downsample_images(original_train_path, train_path)
downsample_images(original_test_path, test_path)

The above functions take all the images in the original train and test set, downsamples them by 4 and saves them. This allowed us to gain computational time.

Segmenting coins from various backgrounds posed significant challenges due to the unique properties and influences of each background on luminance and key features of the coins. For each background, we analyzed characteristics in different color spaces, applied edge detection techniques, and performed extensive testing. Ultimately, we identified a combination of edge detection, thresholding, and other techniques that worked best for each scenario.

We utilized OpenCV for its powerful functions, particularly the Hough Circle Detector to detect coins on clean/thresholded images. 
Each function loads and preprocesses the image, applies the Hough Circle Transform to detect circles, optionally displays the result, and returns a list of detected circles' coordinates and radii.

### Neutral Background
For the neutral background, adaptive thresholding in OpenCV was highly effective. Adaptive thresholding adjusts the threshold dynamically based on the local mean of the neighborhood of a pixel, allowing for better handling of varying lighting conditions. Following this, we applied two morphological operations: a closing (to fill small holes inside the foreground objects) followed by a dilatation (To enhance the coin boundaries).

These steps cleaned the image sufficiently for the fine tuned Hough Circle Detector to accurately detect the coins.

In [ ]:
def detect_and_display_circles_neutral(img, display=False):
    # Load the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur
    blur = cv2.GaussianBlur(gray, (9, 9), 2)

    # Apply adaptive thresholding and morphological operations
    img_th = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    kernel = np.ones((7, 7), np.uint8)
    closing = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)
    closing = cv2.dilate(closing, kernel, iterations=3)

    # Hough Circle Transform
    circles = cv2.HoughCircles(closing, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                               param1=50, param2=10, minRadius=50, maxRadius=120)

    # Convert the (x, y) coordinates and radius of the circles to integers
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]

    # Plot the image with circles
    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            #    circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in circles]
    else:
        return []

### Hand Background
Edge detection was a good solutionfor the hand background. However, the strong lines of the hand persisted in the image. To address this:
1. **Canny Edge Detector**: Applied to highlight the edges of the coins and the hand.
2. **Morphological Closing**: Performed to fill gaps in the detected edges, ensuring that the edges of the coins are continuous.
3. **Line Extraction and Removal**: We used OpenCV's Hough Line extraction function to detect and remove straight lines.
4. **Rectangular Mask**: We added a small rectangular mask at the bottom to cover the watch, which had numerous edges and was challenging to remove.

These steps cleaned the image sufficiently for the fine tuned Hough Circle Detector to accurately detect the coins.

In [ ]:
def detect_and_display_circles_hand(img, display=False):
    # Load the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    edges = cv2.Canny(blurred, 15, 60, apertureSize=3)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=90, minLineLength=1, maxLineGap=80)
    mask = np.ones_like(edges) * 255
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(mask, (x1, y1), (x2, y2), 0, 5)

    edges = cv2.bitwise_and(edges, mask)
    mask_watch = np.ones_like(edges) * 255

    # Mask for the watch
    cv2.rectangle(mask_watch, (0, 850), (1500, 1000), 0, -1)
    edges = cv2.bitwise_and(edges, mask_watch)

    # Apply Hough Circle Transform on the masked image
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                               param1=150, param2=22, minRadius=50, maxRadius=100)

    # Convert the (x, y) coordinates and radius of the circles to integers
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]

    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            #circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles in ')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in circles]
    else:
        return []

## Noisy Background
For the noisy background, no single threshold could effectively separate the coins from the background across different color spaces (YCrCb, RGB, Grayscale, Adaptive Thresholding). Our approach involved:
1. **Superposed Thresholding Masks**: We combined multiple hsv thresholding masks (using an OR operation), each targeting a specific color range of the coins. This method minimized background interference and retained the desired coin colors.
2. **Real-time Thresholding**: We used OpenCV's real-time thresholding function to experiment with different masks and find the optimal thresholds for each category of coins.
3. **Binary Operations**:  We applied two morphological operations to clean the image:
   - **Closing**: To fill small holes inside the foreground objects and ensure the coin boundaries are solid.
   - **Opening**: To remove small noise and extraneous elements from the background.

Finally, we fine-tuned the Hough Circle Detector to achieve coin detection on the cleaned image.

By tailoring our approach to each background, we successfully masked the backgrounds and accurately detected the coins in various challenging conditions.



In [ ]:
def detect_and_display_circles_noisy(img, display=False):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Apply Gaussian Blur to the image
    blur = cv2.GaussianBlur(hsv, (11, 11), 2)
    data_h, data_s, data_v = cv2.split(blur)

    # Apply thresholds to isolate the desired features
    img_th = np.zeros(data_s.shape, dtype=np.uint8)
    # orange coins
    img_th[(data_s > 120) & (data_h < 20)] = 255
    # gray coins
    img_th[(data_s > 82) & (data_h < 23) & (data_s < 170)] = 255
    # for gray bright coins
    img_th[(data_h > 9) & (data_h < 21) & (data_s < 82) & (data_s > 45) & (data_v < 240)] = 255
    img_th[(data_h > 19) & (data_h < 23) & (data_s < 140) & (data_s > 67) & (data_v < 231)] = 255
    # for bright yellow coins
    img_th[(data_h > 19) & (data_v > 210) & (data_s > 153) & (data_h < 25)] = 255

    # Apply morphological operations
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel, iterations=2)
    kernel = np.ones((6, 6), np.uint8)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel, iterations=4)

    # Find contours in the preprocessed image
    circles = cv2.HoughCircles(opening, cv2.HOUGH_GRADIENT, dp=1., minDist=80, param1=200, param2=10, minRadius=40,
                               maxRadius=120)

    # Convert the (x, y) coordinates and radius of the circles to integers
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]

    # Plot the image with circles
    if display:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if circles is not None:
            #    circles = np.uint16(np.around(circles))[0, :]
            for (x, y, r) in circles:
                plt.gca().add_patch(plt.Circle((x, y), r, color='red', fill=False, linewidth=2))
        plt.title('Detected Circles')
        plt.axis('off')
        plt.show()

    if circles is not None:
        return [(x, y, r) for (x, y, r) in circles]
    else:
        return []

### Interactive HSV Threshold Tuning

We used a function that creates an interactive window with trackbars for adjusting HSV values, allowing real-time tuning and visualization of color thresholding on an image. It displays the thresholded output and prints the current HSV ranges when changes are detected.

In the following two images, you can see how it was used to find the thresholds:
<div style="display: flex; justify-content: space-around;">
    <img src="images_rapport/orange.PNG" alt="HSV Threshold Tuning - Image 1" style="width: 45%;"/>
    <img src="images_rapport/gris.PNG" alt="HSV Threshold Tuning - Image 2" style="width: 45%;"/>
</div>



The thresholds weren't perfect, and some noise persisted despite our precision. However, morphological operations resolved this issue.

### Function Descriptions

These functions were utilized for classification and processing tasks later on. They are designed to be straightforward and self-explanatory.

- **crop_coins(image, circles)**: Crops circular regions from the given image based on the provided circle coordinates and radii. Ensures the cropped areas stay within image bounds and returns a list of cropped images.

- **detect_circles_classification(img, display=False)**: Detects circles in the given image using the Hough Circle Transform. Optionally displays the detected circles on the image. Returns the coordinates and radius of the first detected circle.

- **detect_and_crop_coins(image_type, img=None, img_path=None, display_cropped=False)**: Detects and crops coins from the image based on the specified image_type. Loads the image, detects circles using the appropriate function, crops the detected coins, and optionally displays the cropped images. Returns the cropped images and their circle coordinates.

- **crop_whole_directory(root_path, root_output, img_type, save=True)**: Processes all images in a specified directory depending on the type detects and crops coins, and optionally saves the cropped images to an output directory. Returns a list of all cropped images.

- **process_images_in_directory(directory_path, image_type, display_cropped=True)**: Processes all JPG images in a specified directory, detects circles using the appropriate function, and optionally displays the cropped coin images. Prints the detected circle coordinates for each image.


# 3. CHF/EUR/OOD Classifier
Once our segmentation functions were done, we detected all the coins on each image of the train set and saved the images of the cropped coins into a specific folder.

In [146]:
crop_whole_directory(train_path, cropped_path, 'neutral', save=False)
crop_whole_directory(train_path, cropped_path, 'noisy', save=False)
crop_whole_directory(train_path, cropped_path, 'hand', save=False)

neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
noisy
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand
hand


[([array([[[132, 146, 205],
           [136, 147, 207],
           [137, 148, 208],
           ...,
           [176, 168, 205],
           [176, 168, 205],
           [175, 167, 204]],
   
          [[131, 145, 204],
           [135, 146, 206],
           [137, 147, 207],
           ...,
           [179, 171, 208],
           [178, 170, 207],
           [178, 170, 207]],
   
          [[131, 144, 206],
           [134, 145, 205],
           [135, 146, 204],
           ...,
           [180, 172, 209],
           [181, 173, 210],
           [181, 173, 210]],
   
          ...,
   
          [[123, 143, 208],
           [127, 145, 208],
           [125, 143, 206],
           ...,
           [141, 146, 195],
           [140, 145, 194],
           [140, 145, 194]],
   
          [[123, 143, 208],
           [125, 145, 210],
           [125, 143, 206],
           ...,
           [141, 146, 195],
           [139, 146, 195],
           [141, 146, 195]],
   
          [[124, 143, 211],
        

## Final pipeline

In [132]:
### TO ADAPT ###
"""neutral_downsampled = "preprocessed_data/downsampled/train/neutral"
noisy_downsampled = "preprocessed_data/downsampled/train/noisy"
hand_downsampled = "preprocessed_data/downsampled/train/hand"""""

"""neutral_folder = "data/train/neutral"
noisy_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/noisy"
hand_folder = "C:/Users/rapha/OneDrive/Documents/2. EPFL/Master/MA2/Image Analysis & Pattern Recognition/iapr24-coin-counter/train/hand"""""

all_folder = "data/train"
downsampled_all = "preprocessed_data/downsampled/train"

downsampled_test = "preprocessed_data/downsampled/test"
test_folder = "data/test"

model_path1 = "models/classifier_class3_b6_lr5e4_step7_epoch17_split0.75"
#model_path2 = "models/resnet_b6_lr1e2_step7_epoch8_split0.75"
model_path2 = './models/resnet_b32_lr5e1_step7_epoch8_split0.75'

csv_file = "data/train_labels.csv"
csv_path = "submission3.csv"

Choose folder to process

In [133]:
### TO ADAPT ###
folder = test_folder
downsampled_folder = downsampled_test

Background

In [134]:
images, ids, backgrounds = load_images_and_backgrounds(folder, downsampled_folder)

Processing file L0000000.JPG
Processing file L0000001.JPG
Processing file L0000002.JPG
Processing file L0000003.JPG
Processing file L0000004.JPG
Processing file L0000005.JPG
Processing file L0000006.JPG
Processing file L0000007.JPG
Processing file L0000008.JPG
Processing file L0000009.JPG
Processing file L0000010.JPG
Processing file L0000011.JPG
Processing file L0000012.JPG
Processing file L0000013.JPG
Processing file L0000014.JPG
Processing file L0000015.JPG
Processing file L0000016.JPG
Processing file L0000017.JPG
Processing file L0000018.JPG
Processing file L0000019.JPG
Processing file L0000020.JPG
Processing file L0000021.JPG
Processing file L0000022.JPG
Processing file L0000023.JPG
Processing file L0000024.JPG
Processing file L0000025.JPG
Processing file L0000026.JPG
Processing file L0000027.JPG
Processing file L0000028.JPG
Processing file L0000029.JPG
Processing file L0000030.JPG
Processing file L0000031.JPG
Processing file L0000032.JPG
Processing file L0000033.JPG
Processing fil

Predictions

In [135]:
predictions, subclasses = predict(images, ids, backgrounds, model_path1, model_path2)

L0000000
L0000001
L0000002
L0000003
L0000004
L0000005
L0000006
L0000007
L0000008
L0000009
L0000010
L0000011
L0000012
L0000013
L0000014
L0000015
L0000016
L0000017
L0000018
L0000019
L0000020
L0000021
L0000022
L0000023
L0000024
L0000025
L0000026
L0000027
L0000028
L0000029
L0000030
L0000031
L0000032
L0000033
L0000034
L0000035
L0000036
L0000037
L0000038
L0000039
L0000040
L0000041
L0000042
L0000043
L0000044
L0000045
L0000046
L0000047
L0000048
L0000049
L0000050
L0000051
L0000052
L0000053
L0000054
L0000055
L0000056
L0000057
L0000058
L0000059
L0000060
L0000061
L0000062
L0000063
L0000064
L0000065
L0000066
L0000067
L0000068
L0000069
L0000070
L0000071
L0000072
L0000073
L0000074
L0000075
L0000076
L0000077
L0000078
L0000079
L0000080
L0000081
L0000082
L0000083
L0000084
L0000085
L0000086
L0000087
L0000088
L0000089
L0000090
L0000091
L0000092
L0000093
L0000094
L0000095
L0000096
L0000097
L0000098
L0000099
L0000100
L0000101
L0000102
L0000103
L0000104
L0000105
L0000106
L0000107
L0000108
L0000109
L0000110
L

In [136]:
predictions

{'L0000000': array([2., 1., 0.]),
 'L0000001': array([6., 4., 0.]),
 'L0000002': array([4., 0., 0.]),
 'L0000003': array([4., 3., 1.]),
 'L0000004': array([4., 2., 0.]),
 'L0000005': array([7., 1., 0.]),
 'L0000006': array([6., 0., 2.]),
 'L0000007': array([6., 0., 0.]),
 'L0000008': array([2., 1., 1.]),
 'L0000009': array([1., 3., 0.]),
 'L0000010': array([2., 2., 1.]),
 'L0000011': array([4., 3., 1.]),
 'L0000012': array([3., 3., 0.]),
 'L0000013': array([1., 3., 1.]),
 'L0000014': array([4., 1., 0.]),
 'L0000015': array([1., 4., 0.]),
 'L0000016': array([5., 1., 0.]),
 'L0000017': array([5., 2., 0.]),
 'L0000018': array([2., 3., 0.]),
 'L0000019': array([2., 1., 0.]),
 'L0000020': array([2., 2., 2.]),
 'L0000021': array([1., 2., 0.]),
 'L0000022': array([2., 4., 0.]),
 'L0000023': array([6., 4., 1.]),
 'L0000024': array([3., 2., 0.]),
 'L0000025': array([1., 2., 0.]),
 'L0000026': array([2., 4., 0.]),
 'L0000027': array([4., 2., 0.]),
 'L0000028': array([0., 1., 0.]),
 'L0000029': a

id,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD


In [137]:
subclasses

{'L0000000': array([0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'L0000001': array([0., 1., 1., 1., 2., 0., 1., 0., 2., 0., 1., 0., 1., 0., 0., 0.]),
 'L0000002': array([2., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'L0000003': array([0., 0., 0., 3., 0., 0., 1., 0., 0., 0., 0., 2., 1., 0., 0., 1.]),
 'L0000004': array([0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.]),
 'L0000005': array([1., 0., 1., 1., 3., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 'L0000006': array([1., 1., 2., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]),
 'L0000007': array([0., 0., 3., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'L0000008': array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]),
 'L0000009': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 2., 0., 0., 0., 0., 0.]),
 'L0000010': array([0., 0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.]),
 'L0000011': array([1., 1., 0., 1., 1., 0., 0., 1., 0.

In [138]:
#generate_csv_file(subclasses, csv_path)

CHF predictions

In [129]:
import pandas as pd
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)
#df.drop(df.columns[8:17], axis=1, inplace=True)

selected_rows = df[df['id'].isin(subclasses.keys())]
selected_rows.shape

(81, 17)

In [130]:
# Initialize variables to store total correct predictions and total elements
total_correct = 0
total_elements = 0
all_true_labels = []
all_pred_labels = []

# Iterate over each key in subclasses
for key in subclasses.keys():
    # Extract the row values from the DataFrame
    row_values = df.loc[df['id'] == key].iloc[0, 1:17].tolist()

    # Count the number of element-wise equal elements
    equal_count = sum(1 for x, y in zip(row_values, subclasses[key]) if x == y)

    # Increment total correct and total elements
    total_correct += equal_count
    total_elements += len(subclasses[key])

    print(key)
    print(row_values)
    print(subclasses[key])

    # Collect true labels and predicted labels for F1 score computation
    all_true_labels.append(subclasses[key])
    all_pred_labels.append(row_values)

# Calculate total accuracy
total_accuracy = (total_correct / total_elements) * 100

# Compute F1 score using the custom function
f1 = compute_f1(all_pred_labels, all_true_labels)

print(f"Total accuracy: {total_accuracy:.2f}%")
print(f"F1 score: {f1:.4f}")

L1010277
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0]
[1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 2. 0. 0. 0. 0. 0.]
L1010279
[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
L1010281
[0, 0, 0, 0, 2, 3, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0. 0. 0. 2. 2. 1. 1. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
L1010283
[0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0]
[0. 0. 0. 1. 0. 0. 0. 1. 0. 2. 1. 0. 1. 0. 0. 0.]
L1010287
[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 1, 0]
[0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 2. 1. 1. 1. 0.]
L1010288
[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0]
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 2. 0. 0. 0.]
L1010294
[1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
L1010297
[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
[0. 1. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 0. 0. 0.]
L1010298
[0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 2. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
L1010300
[0, 1, 0, 0, 0, 0, 